In [1]:
from transformers import AutoTokenizer,AutoModel
import torch

2024-06-19 15:16:47.737433: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-19 15:16:48.447345: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-06-19 15:16:48.447434: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-06-19 15:16:48.447443: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
import json

def read_squad(path):
    with open(path, 'rb') as f:
        squad_dict = json.load(f)
    contexts = []
    questions = []
    answers = []

    for group in squad_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']

                if 'plausible_answers' in qa.keys():
                    access = 'plausible_answers'
                else:
                    access = 'answers'
                for answer in qa[access]:

                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer)
    return contexts, questions, answers


train_contexts, train_questions, train_answers = read_squad('squad/train-v2.0.json')
val_contexts, val_questions, val_answers = read_squad('squad/dev-v2.0.json')

In [3]:
# !pip install gensim
import numpy as np
# from gensim.models import Word2Vec
np.random.seed(42)
train_contexts = np.array(train_contexts)
train_questions = np.array(train_questions)
train_answers = np.array(train_answers)

val_contexts = np.array(val_contexts)
val_questions = np.array(val_questions)
val_answers = np.array(val_answers)


train_indices = np.random.choice(len(train_contexts), 5000, replace=False)
val_indices = np.random.choice(len(val_contexts), 500, replace=False)

train_contexts_sampled = train_contexts[train_indices]
train_questions_sampled = train_questions[train_indices]
train_answers_sampled = train_answers[train_indices]

val_contexts_sampled = val_contexts[val_indices]
val_questions_sampled = val_questions[val_indices]
val_answers_sampled = val_answers[val_indices]

In [4]:
train_contexts_sampled = train_contexts_sampled.tolist()
train_questions_sampled = train_questions_sampled.tolist()
train_answers_sampled =train_answers_sampled.tolist()

val_contexts_sampled = val_contexts_sampled.tolist()
val_questions_sampled = val_questions_sampled.tolist()
val_answers_sampled = val_answers_sampled.tolist()

In [5]:
def add_end_idx(answers, contexts):
    for answer, context in zip(answers, contexts):
        gold_text = answer['text']
        start_idx = answer['answer_start']
        end_idx = start_idx + len(gold_text)

        if context[start_idx:end_idx] == gold_text:
            answer['answer_end'] = end_idx
        else:
            for n in [1, 2]:
                if context[start_idx-n:end_idx-n] == gold_text:
                    answer['answer_start'] = start_idx - n
                    answer['answer_end'] = end_idx - n

add_end_idx(train_answers_sampled, train_contexts_sampled)
add_end_idx(val_answers_sampled, val_contexts_sampled)


In [6]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
train_encodings = tokenizer(train_contexts_sampled, train_questions_sampled, truncation=True, padding=True)
val_encodings = tokenizer(val_contexts_sampled, val_questions_sampled, truncation=True, padding=True)

In [7]:
from transformers import DistilBertForQuestionAnswering, AdamW
model = DistilBertForQuestionAnswering.from_pretrained('distilbert-base-uncased')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to

In [8]:
def add_token_positions(encodings, answers):
    start_positions = []
    end_positions = []
    for i in range(len(answers)):
        start_pos = encodings.char_to_token(i, answers[i]['answer_start'])
        end_pos = encodings.char_to_token(i, answers[i]['answer_end'])

        if start_pos is None:
            start_pos = tokenizer.model_max_length
        if end_pos is None:
            shift = 1
            while end_pos is None and answers[i]['answer_end'] - shift >= 0:
                end_pos = encodings.char_to_token(i, answers[i]['answer_end'] - shift)
                shift += 1
        if end_pos is None:
            end_pos = tokenizer.model_max_length

        start_positions.append(start_pos)
        end_positions.append(end_pos)
        
    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

add_token_positions(train_encodings, train_answers_sampled)
add_token_positions(val_encodings, val_answers_sampled)


In [9]:
import torch

class SquadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

train_dataset = SquadDataset(train_encodings)
val_dataset = SquadDataset(val_encodings)

In [10]:
from torch.utils.data import DataLoader
from transformers import AdamW
from tqdm import tqdm
import torch
# setup GPU/CPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
if torch.cuda.is_available():
    print("Running on GPU:", torch.cuda.get_device_name(0))
else:
    print("Running on CPU")

Running on GPU: NVIDIA GeForce GTX 1080 Ti


In [11]:
model.to(device)
model.train()
optim = AdamW(model.parameters(), lr=5e-5)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

for epoch in range(3):
    model.train()
    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask,
                        start_positions=start_positions,
                        end_positions=end_positions)
        loss = outputs[0]
        loss.backward()
        optim.step()
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

/home/kunuruabhishek/anaconda3/envs/pytorch/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
Epoch 2: 100%|██████████| 313/313 [02:31<00:00,  2.06it/s, loss=0.804]


In [12]:

model.eval()
val_loader = DataLoader(val_dataset, batch_size=16)
acc = []
for batch in val_loader:
    with torch.no_grad():
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_true = batch['start_positions'].to(device)
        end_true = batch['end_positions'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        start_pred = torch.argmax(outputs['start_logits'], dim=1)
        end_pred = torch.argmax(outputs['end_logits'], dim=1)
        acc.append(((start_pred == start_true).sum()/len(start_pred)).item())
        acc.append(((end_pred == end_true).sum()/len(end_pred)).item())
acc = sum(acc)/len(acc)
acc

0.455078125

In [13]:
import torch
from torch.utils.data import DataLoader

def calculate_f1(pred_start, pred_end, true_start, true_end):
    pred_tokens = set(range(pred_start, pred_end + 1))
    true_tokens = set(range(true_start, true_end + 1))

    common_tokens = pred_tokens.intersection(true_tokens)
    if len(common_tokens) == 0:
        return 0, 0, 0  # precision, recall, f1

    precision = len(common_tokens) / len(pred_tokens)
    recall = len(common_tokens) / len(true_tokens)
    f1 = 2 * precision * recall / (precision + recall)
    return precision, recall, f1


In [14]:
model.eval()

val_loader = DataLoader(val_dataset, batch_size=16)

start_acc = []
end_acc = []
precisions = []
recalls = []
f1s = []

for batch in val_loader:
    with torch.no_grad():
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_true = batch['start_positions'].to(device)
        end_true = batch['end_positions'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        start_pred = torch.argmax(outputs['start_logits'], dim=1)
        end_pred = torch.argmax(outputs['end_logits'], dim=1)
        start_acc.append((start_pred == start_true).sum().item() / len(start_pred))
        end_acc.append((end_pred == end_true).sum().item() / len(end_pred))
        for sp, ep, st, et in zip(start_pred, end_pred, start_true, end_true):
            precision, recall, f1 = calculate_f1(sp.item(), ep.item(), st.item(), et.item())
            precisions.append(precision)
            recalls.append(recall)
            f1s.append(f1)


In [15]:

avg_start_acc = sum(start_acc) / len(start_acc)
avg_end_acc = sum(end_acc) / len(end_acc)
avg_precision = sum(precisions) / len(precisions)
avg_recall = sum(recalls) / len(recalls)
avg_f1 = sum(f1s) / len(f1s)

In [16]:

print(f"Average Start Position Accuracy: {avg_start_acc:.4f}")
print(f"Average End Position Accuracy: {avg_end_acc:.4f}")
print(f"Average Precision: {avg_precision:.4f}")
print(f"Average Recall: {avg_recall:.4f}")
print(f"Average F1 Score: {avg_f1:.4f}")

Average Start Position Accuracy: 0.4453
Average End Position Accuracy: 0.4648
Average Precision: 0.4746
Average Recall: 0.5553
Average F1 Score: 0.4691
